In [ ]:
import numpy as np

import networkx as nx
import matplotlib.pyplot as plt

import os.path as op
import sys
sys.path.append("../")

from src import regmod
from src import utils

## Synthetic model

In [ ]:
path_to_data = "../resources"

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

filename = "adjacency_synthetic.pkl"

adj = utils.load(op.join(path_to_data, filename))
adj -= np.diag(np.diag(adj))

axes[0].imshow(adj, cmap='gray')
axes[0].set_title('Structural connectivity')
toy_graph = nx.Graph(adj)
nx.draw(toy_graph, ax=axes[1], with_labels=True)

In [ ]:
max_path_depth = 2
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
fig.suptitle("Design matrices")

a_design_path = regmod.build_design_paths(adj, max_path_depth)
a_design_degree_row = regmod.build_design_degree_fast(adj, target_deg=False)
a_design_degree_col = regmod.build_design_degree_fast(adj, target_deg=True)

all_designs = [a_design_path, a_design_degree_row, a_design_degree_col]
all_titles = ["Paths", "Degree Row", "Degree Col"]

for mat, axes, title in zip(all_designs, axes, all_titles):
    axes.set_title(title)
    axes.imshow(mat, cmap='gray')

In [ ]:
fig, axes = plt.subplots(figsize=(10, 10))

axes.set_title("Design matix (Paths)")
axes.imshow(a_design_path, cmap='gray', vmin=0, interpolation="none")

## Structural connectome (WM Bundles)

In [ ]:
max_path_depth = 2
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

filename = "adjacency_atlas.pkl"

adj = utils.load(op.join(path_to_data, filename))
adj -= np.diag(np.diag(adj))

axes[0].imshow(adj, cmap='gray')
axes[0].set_title('Structural connectivity')
toy_graph = nx.Graph(adj)
nx.draw(toy_graph, ax=axes[1], with_labels=True)

In [ ]:
max_path_depth = 2
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
fig.suptitle("Design matrices")

a_design_path = regmod.build_design_paths(adj, max_path_depth)
a_design_degree_row = regmod.build_design_degree_fast(adj, target_deg=False)
a_design_degree_col = regmod.build_design_degree_fast(adj, target_deg=True)

all_designs = [a_design_path, a_design_degree_row, a_design_degree_col]
all_titles = ["Paths", "Degree Row", "Degree Col"]

for mat, axes, title in zip(all_designs, axes, all_titles):
    axes.set_title(title)
    axes.imshow(mat, cmap='gray', interpolation="none")

In [ ]:
fig, axes = plt.subplots(figsize=(10, 10))

axes.set_title("Design matix (Paths)")
axes.imshow(a_design_path, cmap='gray', vmin=0, interpolation="none")